## 02 Before You Start

This chapter intends to provide an overview of `Osprey` requirements,
installation guidelines, and helpful suggestions for organizing MRS data in a
meaningful folder structure. The efficient use of `Osprey` requires some very
basic familiarity with MATLAB. If you have never used MATLAB before, someone
else in your group probably has, and can assist with setting everything up for
you to get started.

### Installation

You will need the following software and code:

- **MATLAB** (version 2017a or newer) with the *Optimization Toolbox* and the
  *Statistics Toolbox*. You may need to ask your institutional system
  administrator for a separate license for the toolboxes.

- Download the latest **Osprey** code from our [Github
  repository](https://github.com/schorschinho/osprey), then extract and add
  the entire folder (with subfolders) to your MATLAB path. Make sure to
  regularly check for updates, as we frequently commit new features, bug fixes,
  and improved functions.

- *(Optional)* To perform voxel co-registration and tissue segmentation,
  download **SPM12** [from their
  website](http://www.fil.ion.ucl.ac.uk/spm/software/spm12/), then extract and
  add to your MATLAB path.

- If you want to use the `Osprey` Graphical User Interface (GUI),
  please download the following toolboxes from the MATLAB File Exchange:

    - [GUI Layout
      Toolbox](https://www.mathworks.com/matlabcentral/fileexchange/47982-gui-layout-toolbox)
      (David Sampson)

    - [Widget
      Toolbox](https://www.mathworks.com/matlabcentral/fileexchange/66235-widgets-toolbox)
      (Robin Jackey)

  Download both toolboxes in the MATLAB toolbox format (.mltbx). You can
  double-click to install. MATLAB will automatically add the toolboxes to its
  path.

### Structure and workflow

`Osprey` was designed to have easy workflow with as little user input as
possible. It has many built-in routines to recognize data formats and sequence
types, and will be able to perform most required processing steps automatically.
However, some input will need to be provided by the user in the form of a *job*.

A job specifies the locations of the files containing MRS and structural imaging
data, the type of MRS sequence, and some basic control options over the data
modeling procedure.

While a job is progressing through the `Osprey` pipeline, all raw and processed
data associated with this job are stored in a MATLAB structure, the Osprey
*data container*. By default, this container is called `MRSCont`, but you are free to give it a more meaningful variable name.

`Osprey`'s data handling is based in large parts on the free MATLAB toolbox FID-A. The `Osprey` folder contains a library of, sometimes modified, FID-A functions and additions. Please make sure that you do **not** include an installation of the original FID-A suite in your MATLAB path, as some functions in the `Osprey` folder might be shadowed by FID-A functions with the same name.

The `MRSCont` data container functions as a super-structure, containing
FID-A structures for each dataset and processing step, along with additional
information (e.g. a few basic QA metrics, quantification results, etc).

### How to organize your raw data

Raw MRS data come in an overwhelming variety of formats, each producing
different numbers of files. `Osprey` does not make a lot of assumptions with regard to your folder structure, since you specify the exact location for each file in a job file prior to each analysis. It is **highly** recommended, however, that you store different acquisitions in separate folders. Organizing your raw data in a consistent and meaningful way will save you a lot of time and nerves.

To ensure optimal functioning of `Osprey`, we suggest adapting the folder
hierarchy [proposed by the BIDS (Brain Imaging Data Structure)
initiative](https://github.com/bids-standard/bids-starter-kit/wiki/The-BIDS-folder-hierarchy):

> There are four main levels of the folder hierarchy, these are:
```
> project/
> └── subject
>     └── session
>         └── acquisition
```
> With the exception of the top-level `project` folder, all sub-folders have a specific structure to their name (described below). Here's an example of how this hierarchy looks:
```
> myProject/
> └── sub-01
>     └── ses-01
>         └── anat
```

An example adaptation of this hierarchy for Philips MRS data could look like this:

```
myProject/
└── sub-01 
    └── ses-01
        └── anat
            └──sub-01_T1w.nii
        └── mrs
            └──sub-01_mega-press_act
                └── sub-01_mega-press_act.sdat
                └── sub-01_mega-press_act.spar
            └── sub-01_mega-press_ref
                └── sub-01_mega-press_ref.sdat
                └── sub-01_mega-press_ref.spar
            └── sub-01_press-water
                └──sub-01_press-water_act.sdat
                └──sub-01_press-water_act.spar
└── sub-02 
    └── ses-01
        └── anat
            └──sub-02_T1w.nii
        └── mrs
            └──sub-02_mega-press_act
                └── sub-02_mega-press_act.sdat
                └── sub-02_mega-press_act.spar
            └── sub-02_mega-press_ref
                └── sub-02_mega-press_ref.sdat
                └── sub-02_mega-press_ref.spar
            └── sub-02_press-water
                └──sub-02_press-water_act.sdat
                └──sub-02_press-water_act.spar

```

While adhering to the BIDS standard is a recommendation, it is by no means
binding for most file formats. You can, for example, keep Siemens TWIX files
(`.dat`) all in the same folder, and `Osprey` will be able to handle them, if
you define their paths in the job file accordingly.

**There is one exception: For data stored in single-average DICOM (`*.IMA`, `*.DCM`) or single-average Siemens RDA format, it is absolutely necessary to keep every scan in a separate folder.**

## 03 Quick Start Guide
This chapter is intended to serve as a quick rundown of the `Osprey` workflow, as well as syntax cheat sheet for more experienced users. The full documentation for each of the following commands can be found in their respective sections.

Please familiarize yourself with the basic `Osprey` commands. The number of commands is deliberately kept to a minimum to facilitate a largely automated workflow with as little user interaction as necessary.

In the following example `Osprey` will be used to process Philips MEGA-PRESS data, which is part of the example data. At first `Osprey` will be added to the path.

In [1]:
addpath(genpath('/Users/helge/Documents/GitHub/osprey'));

### OspreyJob

At the beginning of every analysis, you initialize a fresh `MRSCont` data container with a job (defined in the file `jobFile`) using the following command:

```
[MRSCont] = OspreyJob(jobFile);
```

Please find a reduced version of the `jobSDAT_MEGA.m` `jobFile` from the example folder below:

```
jobSDAT_MEGA.m
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%% 1. SPECIFY SEQUENCE INFORMATION %%%

% Specify sequence type
seqType = 'MEGA';               % OPTIONS:    - 'unedited' (default)
                                %             - 'MEGA'
                                %             - 'HERMES'
                                %             - 'HERCULES'
                                
% Specify editing targets
editTarget = {'GABA'};            % OPTIONS:    - 'none' (default if 'unedited')
                                %             - 'GABA', 'GSH'
                                %               (for 'MEGA')
                                %             - 'GABA_GSH', 'GABA_GSH_EtOH'
                                %               (for 'HERMES')
                                %             - 'HERCULES1', 'HERCULES2'
                                %               (for 'HERCULES')
                                
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%% 2. SPECIFY DATA HANDLING AND MODELING OPTIONS %%%

% Save LCModel-exportable files for each spectrum?
opts.saveLCM                = 1;                % OPTIONS:    - 0 (no, default)
                                                %             - 1 (yes)
% Save jMRUI-exportable files for each spectrum?
opts.saveJMRUI              = 1;                % OPTIONS:    - 0 (no, default)
                                                %             - 1 (yes)
                                                
% Save processed spectra in vendor-specific format (SDAT/SPAR, RDA, P)?
opts.saveVendor             = 1;                % OPTIONS:    - 0 (no, default)
                                                %             - 1 (yes)
                                                
% Choose the fitting algorithm
opts.fit.method             = 'LCModel';       % OPTIONS:    - 'Osprey' (default)
                                                %           - 'AQSES' (planned)
                                                %           - 'LCModel' (planned)
                                                %           - 'TARQUIN' (planned)

% Choose the fitting style for difference-edited datasets (MEGA, HERMES, HERCULES)
% (only available for the Osprey fitting method)
opts.fit.style              = 'Separate';   % OPTIONS:  - 'Concatenated' (default) - will fit DIFF and SUM simultaneously)
                                                %           - 'Separate' - will fit DIFF and OFF separately

% Determine fitting range (in ppm) for the metabolite and water spectra
opts.fit.range              = [0.2 4.2];        % [ppm] Default: [0.2 4.2]
opts.fit.rangeWater         = [2.0 7.4];        % [ppm] Default: [2.0 7.4]

% Determine the baseline knot spacing (in ppm) for the metabolite spectra
opts.fit.bLineKnotSpace     = 0.4;              % [ppm] Default: 0.4.

% Add macromolecule and lipid basis functions to the fit? 
opts.fit.fitMM              = 1;                % OPTIONS:    - 0 (no)
                                                %             - 1 (yes, default)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%% 3. SPECIFY MRS DATA AND STRUCTURAL IMAGING FILES %%
% When using single-average Siemens RDA or DICOM files, specify their
% folders instead of single files!

% Specify metabolite data
% (MANDATORY)
files       = {which('exampledata/sdat/sub-01/mrs/sub-01_megapress/sub-01_megapress_act.sdat'),...
               which('exampledata/sdat/sub-02/mrs/sub-02_megapress/sub-02_megapress_act.sdat')};

% Specify water reference data for eddy-current correction (same sequence as metabolite data!)
% (OPTIONAL)
% Leave empty for GE P-files (.7) - these include water reference data by
% default.
files_ref   = {which('exampledata/sdat/sub-01/mrs/sub-01_megapress-ref/sub-01_megapress_ref.sdat'),...
               which('exampledata/sdat/sub-02/mrs/sub-02_megapress-ref/sub-02_megapress_ref.sdat')};

% Specify water data for quantification (e.g. short-TE water scan)
% (OPTIONAL)
files_w     = {which('exampledata/sdat/sub-01/mrs/sub-01_press-ref/sub-01_PRESS_35_ref.sdat'),...
               which('exampledata/sdat/sub-02/mrs/sub-02_press-ref/sub-02_PRESS_35_ref.sdat')};

% Specify T1-weighted structural imaging data
% (OPTIONAL)
% Link to single NIfTI (*.nii) files for Siemens and Philips data
% Link to DICOM (*.dcm) folders for GE data
files_nii   = {which('exampledata/sdat/sub-01/anat/sub-01_T1w.nii'),...
               which('exampledata/sdat/sub-02/anat/sub-02_T1w.nii')};

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%% 4. SPECIFY OUTPUT FOLDER %%
% The Osprey data container will be saved as a *.mat file in the output
% folder that you specify below. In addition, any exported files (for use
% with jMRUI, TARQUIN, or LCModel) will be saved in sub-folders.

% Specify output folder
% (MANDATORY)
outputFolder = strrep(which('exampledata/sdat/jobSDAT_MEGA.m'),'jobSDAT_MEGA.m','derivatives');

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
```

Please refer to the `OspreyLoad` chapter for details on the supported file
formats and the coil combination techniques.

Now we will initilaize a `MRSCont` from the example data:

In [2]:
[MRSCont] = OspreyJob('/Users/helge/Documents/GitHub/osprey/exampledata/sdat/jobSDAT_MEGA.m')


MRSCont = 

  struct with fields:

           flags: [1x1 struct]
            opts: [1x1 struct]
       ospFolder: '/Users/helge/Documents/GitHub/osprey'
           files: {1x2 cell}
       files_ref: {1x2 cell}
         files_w: {1x2 cell}
       files_nii: {1x2 cell}
    outputFolder: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/derivatives'
        colormap: [1x1 struct]
       loadedJob: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/jobSDAT_MEGA.m'
      outputFile: 'jobSDAT_MEGA.mat'



`OspreyJob` will add all necessary information required for data analysis to
the data container, and perform a few basic sanity checks.

A valid Osprey job file contains:
- a list of MRS (and, optionally, structural imaging) data files to be loaded
- basic information on the MRS sequence used
- settings for data modeling
- an output folder to store the results and exported files
- an external csv file with statistical information (e.g. group allocations, age,...)

Osprey job files are described in `.m` format. Please refer to the `OspreyJob` chapter for detailed instructions on how to create a job. A good way to understand the structure of a job file is to study the example jobs in the `Osprey/exampledata` directory, and to create your own job starting from there.

Before you create your own job, consider the recommendations on how to organize your data in the previous chapter.

### OspreyLoad

After loading a job file, you can start the data loading by invoking the
function

```
[MRSCont] = OspreyLoad(MRSCont);
```

`OspreyLoad` will automatically determine the file format, load the raw data,
and, if multi-array receiver coils have been used for acquisition, perform the
receiver-channel combination.

Please refer to the `OspreyLoad` chapter for details on the supported file
formats and the coil combination techniques.

Now we will load the data into MRS container `MRSCont`:

In [3]:
[MRSCont] = OspreyLoad(MRSCont)

All provided datafiles are of the Philips SDAT format.
Loading raw data from dataset 1 out of 2 total datasets...
Loading raw data from dataset 2 out of 2 total datasets...
... done.
Elapsed time is 0.654434 seconds.
Data type Philips SDAT is already coil-combined.

MRSCont = 

  struct with fields:

           flags: [1x1 struct]
            opts: [1x1 struct]
       ospFolder: '/Users/helge/Documents/GitHub/osprey'
           files: {1x2 cell}
       files_ref: {1x2 cell}
         files_w: {1x2 cell}
       files_nii: {1x2 cell}
    outputFolder: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/derivatives'
        colormap: [1x1 struct]
       loadedJob: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/jobSDAT_MEGA.m'
      outputFile: 'jobSDAT_MEGA.mat'
          vendor: 'Philips'
        datatype: 'SDAT'
       nDatasets: 2
             raw: {[1x1 struct]  [1x1 struct]}
         raw_ref: {[1x1 struct]  [1x1 struct]}
           raw_w: {[1x1 struct]  [1x1 struct]}



### OspreyProcess

`OspreyLoad` only loads the raw data and organizes them in a uniform way, in
order to streamline further processing. The next command is a cornerstone of the
entire Osprey pipeline, as it will initiate a series of automatic processing
steps:

```
[MRSCont] = OspreyProcess(MRSCont);
```

`OspreyProcess` performs all state-of-the-art MRS post-processing techniques
to ensure optimal spectral quality, including eddy-current correction,
frequency-and-phase correction, removal of residual water, baseline correction,
and automated phasing and frequency referencing. In addition, several basic
quality metrics (linewidth, SNR) are determined.

`OspreyProcess` is also responsible for handling all aspects of data
processing that are specific to *J*-difference-edited ("spectral editing")
experiments, such as MEGA, HERMES, and HERCULES. These additional steps include
alignment of sub-spectra to reduce subtraction artefacts, as well as the
calculation of (Hadamard-encoded) difference spectra.

Finally, `OspreyProcess` can write the processed spectra into files formats that are readable by spectral fitting software packages like LCModel, TARQUIN, or jMRUI:
- To write LCModel-readable `.RAW` files, you need to set a switch `opts.saveLCM = 1` in your job file. You can then load the `.RAW` file via the `Other` option in the LCModel file type selection menu.
- To write jMRUI/TARQUIN-readable `.TXT` files, you need to set a switch `opts.saveJMRUI = 1` in your job file.
- Please be aware that LCModel may prompt you to enter the number of FID data points (a positive integer number, e.g. 2048), dwell time (in seconds, e.g. 0.0005), and the static magnetic field strength in Hz (eg 123.26 for a Siemens 2.89 T magnet). In TARQUIN, you will probably need to enter the echo time in seconds (e.g. 0.03 for a 30-ms acquisition). You can extract this information from the Osprey `MRSCont` data container, or ...
- ... by setting the switch `opts.saveVendor = 1` in your job file, `Osprey` will create single vendor-specific files, i.e. `.SDAT/.SPAR` files for Philips data, and `.RDA` files for Siemens data, regardless of the raw data format. This function has been introduced to sidestep the above-mentioned problems with `.RAW` and `.TXT` files. LCModel, TARQUIN and jMRUI accept the `.SDAT/.SPAR` and `.RDA` file formats, and should be able to read the header information correctly.

All exported third-party format files are stored in sub-directories of the output folder specified in the job file.

Please refer to the `OspreyProcess` chapter for details on the implementation
of the various post-processing steps.

Now we will process the data from the MRS container `MRSCont`:

In [4]:
[MRSCont] = OspreyProcess(MRSCont)

Processing data from dataset 1 out of 2 total datasets...
Processing data from dataset 2 out of 2 total datasets...
... done.
Elapsed time is 26.266983 seconds.

MRSCont = 

  struct with fields:

           flags: [1x1 struct]
            opts: [1x1 struct]
       ospFolder: '/Users/helge/Documents/GitHub/osprey'
           files: {1x2 cell}
       files_ref: {1x2 cell}
         files_w: {1x2 cell}
       files_nii: {1x2 cell}
    outputFolder: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/derivatives'
        colormap: [1x1 struct]
       loadedJob: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/jobSDAT_MEGA.m'
      outputFile: 'jobSDAT_MEGA.mat'
          vendor: 'Philips'
        datatype: 'SDAT'
       nDatasets: 2
             raw: {[1x1 struct]  [1x1 struct]}
         raw_ref: {[1x1 struct]  [1x1 struct]}
           raw_w: {[1x1 struct]  [1x1 struct]}
       processed: [1x1 struct]
              QM: [1x1 struct]
            info: [1x1 struct]



### OspreyCoreg

You can provide a T1-weighted structural image in NIfTI format (`*.nii`) in
order to perform voxel co-registration and segmentation. The function
`OspreyCoreg` will extract voxel geometry information from the MRS data
headers, and leverage SPM12 NIfTI handling functions to create an SPM image
volume containing a voxel mask. The voxel masks will be saved in a sub-directory (`/VoxelMasks/`) of the output folder specified in the job file.

The path to the structural image is provided in the job definition. After
running `OspreyLoad`, you can call the co-registration routine using the
command

```
[MRSCont] = OspreyCoreg(MRSCont);
```

Please refer to the `OspreyCoreg` chapter for details.

Now we will perform the module on the data from the MRS container `MRSCont`:

In [5]:
[MRSCont] = OspreyCoreg(MRSCont)

Coregistering voxel from dataset 1 out of 2 total datasets...
Coregistering voxel from dataset 2 out of 2 total datasets...
... done.
Elapsed time is 4.817827 seconds.

MRSCont = 

  struct with fields:

           flags: [1x1 struct]
            opts: [1x1 struct]
       ospFolder: '/Users/helge/Documents/GitHub/osprey'
           files: {1x2 cell}
       files_ref: {1x2 cell}
         files_w: {1x2 cell}
       files_nii: {1x2 cell}
    outputFolder: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/derivatives'
        colormap: [1x1 struct]
       loadedJob: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/jobSDAT_MEGA.m'
      outputFile: 'jobSDAT_MEGA.mat'
          vendor: 'Philips'
        datatype: 'SDAT'
       nDatasets: 2
             raw: {[1x1 struct]  [1x1 struct]}
         raw_ref: {[1x1 struct]  [1x1 struct]}
           raw_w: {[1x1 struct]  [1x1 struct]}
       processed: [1x1 struct]
              QM: [1x1 struct]
            info: [1x1 struct]
           

### OspreySeg

`OspreySeg` calls the SPM12 tissue segmentation algorithm ("New
Segment") to segment the T1-weighted structural image into grey matter (GM),
white matter (WM), and cerebro-spinal fluid (CSF). The relative volume fractions
for each tissue class is subsequently used to perform water-scaled concentration
estimation, including CSF correction and tissue-specific relaxation correction.

`OspreySeg` requires that `OspreyLoad` and `OspreyCoreg` have been run before. It can be called with the command

```
[MRSCont] = OspreySeg(MRSCont);
```

Please refer to the `OspreySeg` chapter for details.

Now we will perform the module on the data from the MRS container `MRSCont`:

In [6]:
[MRSCont] = OspreySeg(MRSCont)

Segmenting structural image from dataset 1 out of 2 total datasets...
Segmenting structural image from dataset 2 out of 2 total datasets...
... done.
Elapsed time is 4.413547 seconds.

MRSCont = 

  struct with fields:

           flags: [1x1 struct]
            opts: [1x1 struct]
       ospFolder: '/Users/helge/Documents/GitHub/osprey'
           files: {1x2 cell}
       files_ref: {1x2 cell}
         files_w: {1x2 cell}
       files_nii: {1x2 cell}
    outputFolder: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/derivatives'
        colormap: [1x1 struct]
       loadedJob: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/jobSDAT_MEGA.m'
      outputFile: 'jobSDAT_MEGA.mat'
          vendor: 'Philips'
        datatype: 'SDAT'
       nDatasets: 2
             raw: {[1x1 struct]  [1x1 struct]}
         raw_ref: {[1x1 struct]  [1x1 struct]}
           raw_w: {[1x1 struct]  [1x1 struct]}
       processed: [1x1 struct]
              QM: [1x1 struct]
            info: [1x1 str

### OspreyFit

`OspreyFit` performs linear-combination modeling of the processed spectra.
The Osprey fitting algorithm is using metabolite basis functions for the
specific MRS sequence, which are calculated from 2-D spatially resolved full
density-matrix simulations, using the real sequence timings and pulse waveforms
wherever possible.

The linear-combination modeling performed by `OspreyFit` is similar to the
fitting carried out by the algorithms in LCModel, TARQUIN, or QUEST/AQSES in
jMRUI. In all brevity, the algorithm performs a non-linear least-squares
optimization (Levenberg-Marquardt) of several amplitude, phase, lineshape, and
baseline parameters in order to minimize the difference between the modeled
spectra and the data.

`OspreyFit` will save all relevant modeling parameters into the `MRSCont` data
container. These values will be subsequently used in `OspreyQuantify` to produce
quantitative estimates for all metabolites specified in the basis set.

`OspreyFit` requires that `OspreyLoad` and `OspreyProcess` have been run before. It can be called with the command

```
[MRSCont] = OspreyFit(MRSCont);
```

Please refer to the `OspreyFit` chapter for details on the algorithmic
implementation, basis set creation, and available modeling settings.

Now we will perform the module on the data from the MRS container `MRSCont`:

In [7]:
[MRSCont] = OspreyFit(MRSCont)


Fitting metabolite spectra from dataset 1 out of 2 total datasets...
Running initial referencing...
Running preliminary analysis with reduced basis set...
Running final preliminary analysis step with full basis set...
Running initial referencing...
Running preliminary analysis with reduced basis set...
Running final preliminary analysis step with full basis set...

Fitting metabolite spectra from dataset 2 out of 2 total datasets...
Running initial referencing...
Running preliminary analysis with reduced basis set...
Running final preliminary analysis step with full basis set...
Running initial referencing...
Running preliminary analysis with reduced basis set...
Running final preliminary analysis step with full basis set...
... done.
Elapsed time is 168.026339 seconds.

Fitting water reference from dataset 1 out of 2 total datasets...

                                                 Norm   Rel norm   First-order   Rel first-order      Largest                              Comment
  I

### OspreyQuantify

`OspreyQuantify` calculates various quantitative results from the modeling parameters
that were previously estimated by `OspreyFit`. Depending on the data that you have provided
in the job file (and on the type of sequence that has been run), not all of the following
quantitative results may be available:

- Metabolite ratios with respect to total creatine. Since the internal creatine standard is
present in all spectra, this result is always calculated.
- Raw water-scaled metabolite ratios. This is the pure, uncorrected ratio of the metabolite
amplitude to the amplitude of an unsuppressed water signal. If a short-TE water reference spectrum
has been provided (in the `files_w` field in the job file), its amplitude will be used as the denominator
instead of the water reference signal (`files_ref` in the job file).
- CSF-corrected water-scaled ratios. This is the raw water-scaled metabolite ratio corrected by the fractional
volume of CSF in the value ($1/f_{CSF}$).
- Tissue- and relaxation-corrected water-scaled ratios. These values are fully corrected for tissue-fraction and relaxation effects according to Gasparovic et al, Magn Reson Med 55:1219-1226 (2016).

`OspreyQuantify` requires that at least `OspreyLoad`, `OspreyProcess` and `OspreyFit` have been run before - any metrics involving tissue fractions require `OspreyCoreg` and `OspreySeg` as well. `OspreyQuantify` is invoked with the command

```
[MRSCont] = OspreyQuantify(MRSCont);
```

Please refer to the `OspreyQuantify` chapter for details on the availability of each quantification technique, detailed mathematical implementations, and the default relaxation correction values.

Now we will perform the module on the data from the MRS container MRSCont:

In [8]:
[MRSCont] = OspreyQuantify(MRSCont)

Quantifying dataset 1 out of 2 total datasets...
Quantifying dataset 2 out of 2 total datasets...
... done.
Elapsed time is 0.049987 seconds.

MRSCont = 

  struct with fields:

           flags: [1x1 struct]
            opts: [1x1 struct]
       ospFolder: '/Users/helge/Documents/GitHub/osprey'
           files: {1x2 cell}
       files_ref: {1x2 cell}
         files_w: {1x2 cell}
       files_nii: {1x2 cell}
    outputFolder: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/derivatives'
        colormap: [1x1 struct]
       loadedJob: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/jobSDAT_MEGA.m'
      outputFile: 'jobSDAT_MEGA.mat'
          vendor: 'Philips'
        datatype: 'SDAT'
       nDatasets: 2
             raw: {[1x1 struct]  [1x1 struct]}
         raw_ref: {[1x1 struct]  [1x1 struct]}
           raw_w: {[1x1 struct]  [1x1 struct]}
       processed: [1x1 struct]
              QM: [1x1 struct]
            info: [1x1 struct]
           coreg: [1x1 struct]
     

## OspreyOverview

`OspreyOverview` creates scaled individual spectra for all sub-spectra. Furthermore, the indivdual fits, baseline estimates, and residuals are stored. For each of the before mentioned measures the mean and the standard deviation are calculated.
If different groups are passed through the external csv file the data is separated based on the corresponding groups.

`OspreyOverview` requires that at least `OspreyLoad`, `OspreyProcess` and `OspreyFit` have been run before. `OspreyOverview` is invoked with the command

```
[MRSCont] = OspreyOverview(MRSCont);
```

Please refer to the `OspreyOverview` chapter for details on the scaling and how correlation measures are stored.

Now we will perform the module on the data from the MRS container MRSCont:

In [9]:
[MRSCont] = OspreyOverview(MRSCont)


MRSCont = 

  struct with fields:

           flags: [1x1 struct]
            opts: [1x1 struct]
       ospFolder: '/Users/helge/Documents/GitHub/osprey'
           files: {1x2 cell}
       files_ref: {1x2 cell}
         files_w: {1x2 cell}
       files_nii: {1x2 cell}
    outputFolder: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/derivatives'
        colormap: [1x1 struct]
       loadedJob: '/Users/helge/Documents/GitHub/osprey/exampledata/sdat/jobSDAT_MEGA.m'
      outputFile: 'jobSDAT_MEGA.mat'
          vendor: 'Philips'
        datatype: 'SDAT'
       nDatasets: 2
             raw: {[1x1 struct]  [1x1 struct]}
         raw_ref: {[1x1 struct]  [1x1 struct]}
           raw_w: {[1x1 struct]  [1x1 struct]}
       processed: [1x1 struct]
              QM: [1x1 struct]
            info: [1x1 struct]
           coreg: [1x1 struct]
             seg: [1x1 struct]
             fit: [1x1 struct]
        quantify: [1x1 struct]
        overview: [1x1 struct]



### OspreyGUI

At any given point during your analysis, you can invoke the `Osprey` GUI to visualize the above steps.

The `Osprey` GUI contains several tabs representing the various stages of the `Osprey` workflow, i.e. `Load`, `Process`, `Fit`, `Coreg/Seg`, and `Quantify`. It allows the export of different figures as esp files.

In addition, various useful summary plots and basic statistical evaluations of the quantitative results are shown in the `Overview` tab. This tab gets activated only if more than one dataset has been fully analyzed.

The GUI can be initialized using the command.

```
OspreyStartUp
```

All of the above-mentioned analysis steps can also be called from inside the GUI using the respective buttons. Please refer to the `OspreyGUI` chapter for details.

You can also load a `MRSCont` from your MATLAB workspace into the GUI by using

```
gui = OspreyGUIapp(MRSCont);
```